<a href="https://colab.research.google.com/github/Monokhan/Analisis-de-transporte-logistico-con-Python/blob/main/Analisis_de_transporte_logistico_Verno_Feliz_con_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias

# Librerias y Datos

In [7]:
!pip install openpyxl
import pandas as pd
import numpy as np
from folium import features
from gurobipy import *

#The URL was pointing to a webpage, not the raw file. Also, the file is an xlsx file and needs to be read in using the openpyxl engine.
excel = pd.ExcelFile('https://github.com/Monokhan/Analisis-de-transporte-logistico-con-Python/raw/main/Verno%20Feliz%20en%20Europa-Coord.xlsx', engine='openpyxl')
df = pd.read_excel(excel, "Sheet1", index_col=0)  # Clientes europa
df2 = pd.read_excel(excel, "Sheet2", index_col=0)  # Centros de distribucion
df3 = pd.read_excel(excel, "Sheet3", index_col=0)  # Distancias entre centro de distribucion y el cliente
df4 = pd.read_excel(excel, "Sheet4", index_col=0)  # Fabricas ubicadas en Chile


In [8]:
df

,Pais,Demanda [unidades],Demanda [Millones],Costo Fijo [$],Costo Variable [ $/ productos],Capacidad [unidades],lat,lon
Ciudad,,,,,,,,
Madrid,España,121,6.550,25472,4,750,40.416775,-3.703790
Paris,Francia,240,2.140,48005,2,800,48.856614,2.352222
Londres,Reino Unido,250,8.900,54705,5,500,51.507351,-0.127758
Liverpool,Reino Unido,121,0.552,45563,4,750,53.408371,-2.991573
Roma,Italia,159,2.870,33375,4,250,41.902783,12.496365
Milan,Italia,198,1.350,48274,5,300,45.464204,9.189982
Zurich,Suiza,282,0.402,21148,6,150,47.376887,8.541694
Berlin,Alemania,211,3.750,54988,4,700,52.520007,13.404954
Bruselas,Bélgica,224,0.174,54231,6,500,50.850340,4.351710


In [9]:
df2

,Pais,Demanda [unidades],Población [Millones],Costo Fijo [$],Costo Variable [ $/ productos],Capacidad [unidades],lat,lon
Ciudad,,,,,,,,
Madrid,España,121,6.550,25472,4,750,40.416775,-3.703790
Paris,Francia,240,2.140,48005,2,800,48.856614,2.352222
Liverpool,Reino Unido,121,0.552,45563,4,750,53.408371,-2.991573
Milan,Italia,198,1.350,48274,5,300,45.464204,9.189982
Berlin,Alemania,211,3.750,54988,4,700,52.520007,13.404954
Bruselas,Bélgica,224,0.174,54231,6,500,50.850340,4.351710
Amsterdam,Hollanda,259,0.821,38114,3,800,52.366697,4.894540
Budapest,Hungría,122,1.750,59699,3,500,47.497912,19.040235
Varsovia,Polonia,175,1.708,54468,1,500,52.229676,21.012229


In [10]:
df3

,Pais,Madrid,Paris,Londres,Liverpool,Roma,Milan,Zurich,Berlín,Bruselas,Ámsterdam,Dublín,Praga,Bratislava,Budapest,Atenas,Varsovia
Ciudad,,,,,,,,,,,,,,,,,
Madrid,España,10.00,1052.97,1263.10,1445.26,1367.60,1190.45,1248.90,1871.43,1316.97,1481.41,1450.64,1775.59,1861.96,1978.45,2375.28,2294.00
Paris,Francia,1052.97,10.00,343.92,629.18,1106.60,640.74,489.17,879.69,264.27,429.87,782.50,885.46,1091.51,1247.68,2099.04,1370.54
Liverpool,Reino Unido,1445.26,629.18,287.43,10.00,1721.78,1246.27,1057.38,1103.69,577.43,543.03,217.61,1256.01,1527.05,1688.43,2668.40,1615.88
Milan,Italia,1190.45,640.74,960.88,1246.27,477.03,10.00,218.37,842.76,698.40,829.29,1418.42,645.88,673.58,788.82,1463.89,1145.92
Berlín,Alemania,1871.43,879.69,934.52,1103.69,1182.35,842.76,669.79,10.00,652.58,578.60,1321.27,281.33,552.95,688.90,1803.12,518.84
Bruselas,Bélgica,1316.97,264.27,321.60,577.43,1173.92,698.40,492.57,652.58,10.00,172.85,777.63,720.75,970.18,1131.76,2091.71,1163.40
Amsterdam,Hollanda,1481.41,429.87,358.31,543.03,1296.71,829.29,613.61,578.60,172.85,10.00,758.48,713.12,987.50,1148.22,2165.63,1097.39
Budapest,Hungría,1978.45,1247.68,1453.28,1688.43,809.07,788.82,791.45,688.90,1131.76,1148.22,1899.97,443.19,161.78,10.00,1123.88,545.00
Varsovia,Polonia,2294.00,1370.54,1453.14,1615.88,1315.68,1145.92,1045.41,518.84,1163.40,1097.39,1832.86,518.41,532.57,545.00,1597.26,10.00


In [11]:
df4

,Costo Fijo,Costo Variable,Capacidad,Costo Envio Europa x unidad
La Serena,20000,1.0,2000,2
Coquimbo,15000,0.8,500,2
Antofagasta,20000,1.5,2000,2
Vallenar,10000,1.0,500,2
Copiapo,25000,1.5,1500,2


## Subíndice

In [12]:
DC = list(df2.index)
ciudad = list(df.index)
fabrica=list(df4.index)


## Grafica  de ubicaciones

In [13]:
#importar folium, si no esta instalado, instalar en el anaconda
#usar el script, pip install folium
#una vez ya instalado reiniciamos el kernel
#aunq lo tenia importado mas arriba igual me tiraba error, por lo cual lo volvi a importar a esta casilla

import folium
from folium import features
mapa = folium.Map(location=[df['lat'].mean(), df['lon'].mean()],zoom_start=4)

fg=folium.FeatureGroup()
for n in range(len(df)):
    fg.add_child(folium.CircleMarker(location=[df.iloc[n][6],df.iloc[n][7]],radius=3,color='#ff0000',fill=True,fill_color='#ff0000'))


    fg.add_child(folium.Marker(location=[df.iloc[n][6],df.iloc[n][7]],
                               popup=folium.Popup(ciudad[n]),
                               icon=folium.Icon(color='green',
                                                icon_color='white',icon='info-sign')))
mapa.add_child(fg)
mapa

<ipython-input-13-b2b9d84bce6b>:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fg.add_child(folium.CircleMarker(location=[df.iloc[n][6],df.iloc[n][7]],radius=3,color='#ff0000',fill=True,fill_color='#ff0000'))
<ipython-input-13-b2b9d84bce6b>:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fg.add_child(folium.Marker(location=[df.iloc[n][6],df.iloc[n][7]],


Arcos

In [14]:
arcos=[(i,j)for i in fabrica for j in ciudad]
#arcos1=
#arcos2=
arcos

[('La Serena', 'Madrid'),
 ('La Serena', 'Paris'),
 ('La Serena', 'Londres'),
 ('La Serena', 'Liverpool'),
 ('La Serena', 'Roma '),
 ('La Serena', 'Milan'),
 ('La Serena', 'Zurich'),
 ('La Serena', 'Berlin'),
 ('La Serena', 'Bruselas'),
 ('La Serena', 'Amsterdam'),
 ('La Serena', 'Dublin'),
 ('La Serena', 'Praga'),
 ('La Serena', 'Bratislava'),
 ('La Serena', 'Budapest'),
 ('La Serena', 'Atenas'),
 ('La Serena', 'Varsovia'),
 ('Coquimbo', 'Madrid'),
 ('Coquimbo', 'Paris'),
 ('Coquimbo', 'Londres'),
 ('Coquimbo', 'Liverpool'),
 ('Coquimbo', 'Roma '),
 ('Coquimbo', 'Milan'),
 ('Coquimbo', 'Zurich'),
 ('Coquimbo', 'Berlin'),
 ('Coquimbo', 'Bruselas'),
 ('Coquimbo', 'Amsterdam'),
 ('Coquimbo', 'Dublin'),
 ('Coquimbo', 'Praga'),
 ('Coquimbo', 'Bratislava'),
 ('Coquimbo', 'Budapest'),
 ('Coquimbo', 'Atenas'),
 ('Coquimbo', 'Varsovia'),
 ('Antofagasta', 'Madrid'),
 ('Antofagasta', 'Paris'),
 ('Antofagasta', 'Londres'),
 ('Antofagasta', 'Liverpool'),
 ('Antofagasta', 'Roma '),
 ('Antofagasta',

# Envios dentro de Europa



In [26]:
import pandas as pd

# Assuming your DataFrame is named 'df' and has columns 'Ciudad' and 'Pais'
# Replace these with your actual column names if they are different

europe_countries = ['Albania', 'Andorra', 'Armenia', 'Austria', 'Azerbaijan', 'Belarus', 'Belgium',
                     'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic',
                     'Denmark', 'Estonia', 'Finland', 'France', 'Georgia', 'Germany', 'Greece',
                     'Hungary', 'Iceland', 'Ireland', 'Italy', 'Kazakhstan', 'Kosovo', 'Latvia',
                     'Liechtenstein', 'Lithuania', 'Luxembourg', 'Malta', 'Moldova', 'Monaco',
                     'Montenegro', 'Netherlands', 'North Macedonia', 'Norway', 'Poland', 'Portugal',
                     'Romania', 'Russia', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain',
                     'Sweden', 'Switzerland', 'Turkey', 'Ukraine', 'United Kingdom', 'Vatican City']

# Filter the DataFrame to include shipments to Europe
shipments_to_europe = df[df['Pais'].apply(lambda x: x in europe_countries)]

# Group by city and get the corresponding country
shipments_by_city = shipments_to_europe.groupby('Ciudad')['Pais'].unique()

# Display the result
print(shipments_by_city)

Series([], Name: Pais, dtype: object)


In [22]:
#Crear modelo Gurobi
m=Model('Verano Feliz en Europa')

Restricted license - for non-production use only - expires 2025-11-24


In [21]:
#para obtener la matriz de distancia
dist={(DC[k], ciudad[j]):df3.iloc[k][j+1] for k in range(len(DC))for j in range(len(ciudad))}
dist

<ipython-input-21-54acd3f8283d>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dist={(DC[k], ciudad[j]):df3.iloc[k][j+1] for k in range(len(DC))for j in range(len(ciudad))}


{('Madrid', 'Madrid'): 10.0,
 ('Madrid', 'Paris'): 1052.97,
 ('Madrid', 'Londres'): 1263.1,
 ('Madrid', 'Liverpool'): 1445.26,
 ('Madrid', 'Roma '): 1367.6,
 ('Madrid', 'Milan'): 1190.45,
 ('Madrid', 'Zurich'): 1248.9,
 ('Madrid', 'Berlin'): 1871.43,
 ('Madrid', 'Bruselas'): 1316.97,
 ('Madrid', 'Amsterdam'): 1481.41,
 ('Madrid', 'Dublin'): 1450.64,
 ('Madrid', 'Praga'): 1775.59,
 ('Madrid', 'Bratislava'): 1861.96,
 ('Madrid', 'Budapest'): 1978.45,
 ('Madrid', 'Atenas'): 2375.28,
 ('Madrid', 'Varsovia'): 2294.0,
 ('Paris', 'Madrid'): 1052.97,
 ('Paris', 'Paris'): 10.0,
 ('Paris', 'Londres'): 343.92,
 ('Paris', 'Liverpool'): 629.18,
 ('Paris', 'Roma '): 1106.6,
 ('Paris', 'Milan'): 640.74,
 ('Paris', 'Zurich'): 489.17,
 ('Paris', 'Berlin'): 879.69,
 ('Paris', 'Bruselas'): 264.27,
 ('Paris', 'Amsterdam'): 429.87,
 ('Paris', 'Dublin'): 782.5,
 ('Paris', 'Praga'): 885.46,
 ('Paris', 'Bratislava'): 1091.51,
 ('Paris', 'Budapest'): 1247.68,
 ('Paris', 'Atenas'): 2099.04,
 ('Paris', 'Varsovia

# Variables de decisión

Xi= 1 cuando se decide utilizar la Fabrica i. 0 de otra manera
Yij = cantidad de productos enviados desde la fabrica i al cliente j

In [23]:
#crear variables de desicion
x=m.addVars(fabrica, name='x',vtype=GRB.BINARY)
y=m.addVars(arcos, name='y',vtype=GRB.CONTINUOUS)

In [ ]:
#crea un grafico que muestre  la cantidad de envios por ciudad durante todo el tiempo registrado
